In [1]:
%env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

env: MINERL_DATA_ROOT=/Volumes/CORSAIR/data


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
from fgz.training.xirl_trainer import XIRLTrainer
from fgz_config import FGZConfig
import torch
import wandb
from tqdm import tqdm

/Users/dyllan/miniconda3/envs/minerl/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.empty_cache()  # fix memory leaks

In [5]:
config = FGZConfig(
    enabled_tasks=[2, 3],  # cave and waterfall 
    disable_fmc_detection=True,  # if true, only classification will occur. 
    use_wandb=True,
)

In [6]:
# minerl_env = gym.make('MineRLBasaltMakeWaterfall-v0')
# agent = get_agent(config)
# dynamics_env = get_dynamics_environment(config)

In [7]:
# agent.device

In [8]:
# dynamics_function_optimizer = torch.optim.Adam(
#     dynamics_env.dynamics_function.parameters(),
#     lr=0.000005,
#     weight_decay=1e-4,
# )

In [9]:
dataset_path = config.dataset_paths[0]
print(dataset_path)
trainer = XIRLTrainer(dataset_path, config.model_path, config.weights_path)

/Volumes/CORSAIR/data/MineRLBasaltFindCave-v0


In [10]:
# import ray
# ray.get(trainer.train_on_pair())
trainer.train_on_pair()

bytes for the pair of trajectories: 6549504
tensor(1.8643, grad_fn=<AddBackward0>)
tensor(2.0136, grad_fn=<AddBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
trainer.get_nbytes_stored()